# Imports

In [64]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import json
from string import punctuation
import pickle
import random
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

# Preprocessing

In [42]:
words, tags, classes, docs = [], [], [], []
ignore = list(punctuation)
lemmatizer = WordNetLemmatizer()

In [43]:
intents = json.loads(open('intents.json').read())

In [52]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        tokenized = nltk.word_tokenize(pattern)
        tokenized = list(map(lambda word: word.lower(), tokenized))
        words.extend(tokenized)
        docs.append((tokenized, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [53]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore]

In [54]:
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [57]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Creating Training Data

In [58]:
training = []
output_empty = [0] * len(classes)

In [59]:
for doc in docs:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [60]:
random.shuffle(training)
X = np.array([item[0] for item in training])
y = np.array([item[1] for item in training])

In [11]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (141, 165)
Shape of y: (141, 27)


# Creating the Model

In [66]:
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [67]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                              verbose=1, restore_best_weights=True)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', verbose=1)

In [68]:
model = Sequential([
    Dense(128, input_shape=(len(X[0]),), activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.5),
    Dense(len(y[0]), activation='softmax')
])

In [69]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [70]:
history = model.fit(X, y, epochs=200)

Epoch 1/200
23/23 [==============================] - 1s 3ms/step - loss: 3.3017 - accuracy: 0.0411
Epoch 2/200
23/23 [==============================] - 0s 2ms/step - loss: 3.2422 - accuracy: 0.0667
Epoch 3/200
23/23 [==============================] - 0s 3ms/step - loss: 3.1756 - accuracy: 0.1234
Epoch 4/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1128 - accuracy: 0.1376
Epoch 5/200
23/23 [==============================] - 0s 2ms/step - loss: 3.0521 - accuracy: 0.1504
Epoch 6/200
23/23 [==============================] - 0s 2ms/step - loss: 2.9859 - accuracy: 0.1645
Epoch 7/200
23/23 [==============================] - 0s 2ms/step - loss: 2.9242 - accuracy: 0.2099
Epoch 8/200
23/23 [==============================] - 0s 2ms/step - loss: 2.8239 - accuracy: 0.2241
Epoch 9/200
23/23 [==============================] - 0s 2ms/step - loss: 2.7617 - accuracy: 0.2312
Epoch 10/200
23/23 [==============================] - 0s 2ms/step - loss: 2.6376 - accuracy: 0.2355
Epoch 11/

In [71]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               25088     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 27)                1755      
                                                                 
Total params: 35099 (137.11 KB)
Trainable params: 35099 (137.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [77]:
model.save('chatbot_model.h5', history)